In [1]:
from twitter import *
import oauth2 as oauth
import requests
import json
import sys
import networkx as nx
import time
import pickle
import collections
import matplotlib.pyplot as plt
import community
from sys import maxsize
from functools import partial

In [2]:
def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw):
    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
        if wait_period > 3600: # Seconds
            print('Too many retries. Quitting.', file=sys.stderr)
            raise e
        if e.e.code == 401:
            return None
        elif e.e.code == 404:
            print('Encountered 404 Error (Not Found)', file=sys.stderr)
            return None
        elif e.e.code == 429:
            print('Encountered 429 Error (Rate Limit Exceeded)', file=sys.stderr)
            if sleep_when_rate_limited:
                print("Retrying in 15 minutes...ZzZ...", file=sys.stderr)
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print('...ZzZ...Awake now and trying again.', file=sys.stderr)
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print('Encountered %i Error. Retrying in %i seconds' % (e.e.code, wait_period), file=sys.stderr)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function

    wait_period = 2
    error_count = 0
    while True:
        try:
            return twitter_api_func(*args, **kw)
        except TwitterHTTPError as e:
            error_count = 0
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except URLError as e:
            error_count += 1
            print("URLError encountered. Continuing.", file=sys.stderr)
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise
        except BadStatusLine as e:
            error_count += 1
            print >> sys.stderr, "BadStatusLine encountered. Continuing."
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise

In [3]:
def getKeysCount():
    with open('auth_dict.json','r') as f:
        twtr_auth = json.load(f) 
    return len(twtr_auth['consumer_key'].split(","))

In [4]:
def graphBuilder(user_name):
    max_twitter_keys = 1
    max_twitter_keys = getKeysCount()
    depth = 3
    G = nx.Graph()
    id_to_name_dict = {} 
    count = 0
    key_counter, key_index = 0,0
    api_hits, cursor  = 0, -1
    rate_limit = 14
    node_count = 1
    window_frame = 900
    timeLag = window_frame / (rate_limit * max_twitter_keys) + 0.05
    bfs_nodes = [user_name]
    G.add_node(user_name)
    stop_flag = True
    while len(bfs_nodes) > 0 and stop_flag: 
        cursor = -1
        curr_user_Name = bfs_nodes[0]
        bfs_nodes = bfs_nodes[1:]
        t = getTwitterHandle(key_index)	
        while cursor != 0:
            if api_hits == rate_limit:
                key_counter = key_counter + 1
                key_index = key_counter % max_twitter_keys
                t = getTwitterHandle(key_index) 
                api_hits = 0
            if node_count < pow(5, depth) + 1: 
                api_hits = api_hits + 1
                top_five_friends = getTopReciprocalFriends(t, curr_user_Name, id_to_name_dict, timeLag)
                cursor = 0 
                bfs_nodes = bfs_nodes + top_five_friends;
                node_count = node_count + 5
                print('Number of nodes in graph : ' + str(node_count))
                for i in top_five_friends:
                    G.add_node(i)
                    G.add_edge(curr_user_Name, i)
            else:
                stop_flag = False
                break
    return G

In [5]:
def getTopReciprocalFriends(t, curr_user_name, id_to_name_dict, timeLag):
    print('Searching top five Reciprocal Friends :' + curr_user_name)
    temp_dict = collections.OrderedDict()
    friends = make_twitter_request(t.friends.ids,screen_name=curr_user_name, count = 5000)
    followers = make_twitter_request(t.followers.ids,screen_name=curr_user_name, count = 5000)
    reciprocal_friends = set(friends['ids']) & set(followers['ids']) 
    time.sleep(timeLag)
    reciFrnsIdList = list(reciprocal_friends)[:100] 
    for i in range(len(reciFrnsIdList)):
        temp = t.users.lookup(user_id=reciFrnsIdList[i], _timeout=None)
        if temp[0]['protected'] != True:
            temp_dict[temp[0]['id']] = temp[0]['followers_count']
    top_five_Ids = [i[0] for i in list(temp_dict.items())[-5:]]
    top_five_names = []
    for i in range(len(top_five_Ids)):
        temp = t.users.lookup(user_id=top_five_Ids[i], _timeout=None)
        id_to_name_dict[temp[0]['id']] = temp[0]['screen_name']
        top_five_names.append(temp[0]['screen_name']);
    return top_five_names

In [6]:
def getTwitterHandle(index):
    with open('auth_dict.json','r') as f:
        twtr_auth = json.load(f)
    try:
        consumer_key_list = twtr_auth['consumer_key'].split(",")
        consumer_secret_key_list = twtr_auth['consumer_secret'].split(",")
        token_key_list = twtr_auth['token'].split(",")
        token_secret_list = twtr_auth['token_secret'].split(",")
        consumer_key = consumer_key_list[index]
        consumer_secret_key = consumer_secret_key_list[index]
        token_key = token_key_list[index]
        token_secret = token_secret_list[index]
        t = Twitter(auth=OAuth(token_key,token_secret, consumer_key, consumer_secret_key))
        
        return t
    except:
        e = sys.exc_info()[0]
        print('Exception caught: ' + str(e))
        return None

In [ ]:
G = graphBuilder('NimishPatankar') # add user name as argument
plt.figure(figsize=(10, 10))
plt.axis('off')
parts = community.best_partition(G)  
position = nx.spring_layout(G)  
nx.draw_networkx_nodes(G, position, node_size=100, cmap=plt.cm.RdYlBu, node_color=list(parts.values()))
nx.draw_networkx_edges(G, position, alpha=0.5)
plt.show(G)


Searching top five Reciprocal Friends :NimishPatankar
Number of nodes in graph : 6
Searching top five Reciprocal Friends :282singhVivek
Number of nodes in graph : 11
Searching top five Reciprocal Friends :Atharva_Garate
Number of nodes in graph : 16
Searching top five Reciprocal Friends :bhumijoshi2
Number of nodes in graph : 21
Searching top five Reciprocal Friends :Bombayduck10
Number of nodes in graph : 26
Searching top five Reciprocal Friends :MehulSuhan
Number of nodes in graph : 31
Searching top five Reciprocal Friends :aliveway
Number of nodes in graph : 36
Searching top five Reciprocal Friends :NimishPatankar
Number of nodes in graph : 41
Searching top five Reciprocal Friends :bubblegurrl
Number of nodes in graph : 46
Searching top five Reciprocal Friends :MmmSexFacts
